# Basics of Digitization
*Mikołaj Leszczuk, Jakub Nawała, Grzegorz Pasterczyk*


## Introduction
One of the basic tasks in processing and transmitting video signals is signal conversion. In particular, signals are converted from the analogue to digital form. Sometimes, at the same processing stage, visual material acquired is compressed “on-the-fly”. Then it is stored using one of the popular container formats like MP4 or AVI. This fast, “on-the-fly” compression may not be sufficient for some applications. Thus, further recompression is sometimes necessary (e.g., to fit into predefined streaming bandwidth or satisfy maximum file size requirements).<br>
Viewing (i.e., playing) a compressed video requires decompression (most often in real time). Afterwards, moving pictures are transmitted onto a screen (in a digital or, after a conversion, in an analogue form). The processes of digitization and viewing will be performed throughout this exercise.

## Purpose of the Exercise 
The purpose of this exercise is to larn about digitization of analogue video signals (for example, as recorded with your built-in webcam)

Simultaneously, you will learn how to play-out the previously created file. For digitization purposes, the built-in webcam of your computer and this Jupyter Notebook will be used.

## Execution of the Exercise
Please read and execute the whole instruction. The exercise stand should be equipped with a computer having a monitor, camera and an internet connection.

## Setting Up the Computer
The first time you use this notebook you will be asked to allow it to access your camera.
Please allow the notebook to access the camera to make sure everything works as expected.

In [ ]:
#@title <=== Please click and wait several seconds to prepare the software environment { display-mode: "form" }
!apt-get install libzbar0
!pip3 install pyzbar
import matplotlib.pyplot as pyplot
import cv2
import numpy as np
from ipywidgets import interact, IntSlider
import pyzbar.pyzbar as pyzbar
import urllib.request, urllib.error
import os.path
from google.colab import files
!git clone https://github.com/gpasterczyk/digitization_1

def readImage(path):
    if os.path.isfile(path):
        cap = cv2.imread(path)
        oryg = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)
        return oryg

    else:
        try:
            response = urllib.request.urlopen(path)
            code = 200
        except urllib.error.URLError as e:
            code = 'Name resolve problem'
        except urllib.error.HTTPError as e:
            code = e.code
        if code == 200:
            req = urllib.request.urlopen(urllib.request.Request(path, headers={'User-Agent': 'Mozilla/82.0'}))
            arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
            cap = cv2.imdecode(arr, -1)
            oryg = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)
            return oryg
        else:
            print(code)
            #cap = cv2.imread('digitization_1/files/http_error.png')
            #oryg = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)
            #return oryg
        print(code)

def ploting2(toPlot1, title1, toPlot2, title2, cmap):
    pyplot.figure(figsize=[15.0, 5.0])
    pyplot.subplot(1,2,1)
    pyplot.title(title1)
    pyplot.imshow(toPlot1)
    pyplot.axis('off')
    pyplot.subplot(1,2,2)
    pyplot.title(title2)
    pyplot.imshow(toPlot2, cmap)
    pyplot.axis('off')
    pyplot.show()

def cs(image, set_channel):
    oryg = readImage(image)
    title1 = 'Original image'
    title2 = 'Channel {} of original image'.format(set_channel)
    cmap = 'viridis'

    if set_channel in {'R','G','B'}:
        h, w = oryg.shape[0], oryg.shape[1]
        zeros = np.zeros((h,w), dtype="uint8")
        r, g, b = cv2.split(oryg)
        if set_channel == 'R':
            typ = cv2.merge((r,zeros,zeros))
        elif set_channel == 'G':
            typ = cv2.merge((zeros,g,zeros))
        elif set_channel == 'B':
            typ = cv2.merge((zeros,zeros,b))


    elif set_channel in {'Y','Cr','Cb'}:
        frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2YCrCb)
        cmap = 'gray'
        if set_channel == 'Y':
            typ = cv2.split(frame)[0]
        elif set_channel == 'Cb':
            typ = cv2.split(frame)[2]
        elif set_channel == 'Cr':
            typ = cv2.split(frame)[1]


    elif set_channel in {'Y_','U','V'}:
        frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2YUV)
        cmap = 'gray'
        if set_channel == 'Y_':
            typ = cv2.split(frame)[0]
        elif set_channel == 'U':
            typ = cv2.split(frame)[1]
        elif set_channel == 'V':
            typ = cv2.split(frame)[2]


    elif set_channel in {'H','S','V_'}:
        frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2HSV)
        cmap = 'gray'
        if set_channel == 'H':
            typ = cv2.split(frame)[0]
        elif set_channel == 'S':
            typ = cv2.split(frame)[1]
        elif set_channel == 'V_':
            typ = cv2.split(frame)[2]


    elif set_channel in {'RGB','YCbCr','YUV','HSV'}:
        title2 = 'Original image in {} color space'.format(set_channel)
        if set_channel == 'RGB':
            typ = oryg
        elif set_channel == 'YCbCr':
            frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2YCrCb)
            cap = cv2.cvtColor(frame, cv2.COLOR_YCrCb2RGB)
            typ = cap
        elif set_channel == 'YUV':
            frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2YUV)
            cap = cv2.cvtColor(frame, cv2.COLOR_YUV2RGB)
            typ = cap
        elif set_channel == 'HSV':
            frame = cv2.cvtColor(oryg, cv2.COLOR_RGB2HSV)
            cap = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)
            typ = cap
    ploting2(oryg, title1, typ, title2, cmap)

def histogram(image):
    oryg = readImage(image)
    h, w = oryg.shape[0], oryg.shape[1]
    print('Loaded image size is {}x{}'.format(w, h))
    color = ('r','g','b')
    pyplot.figure(figsize=[15.0, 5.0])
    pyplot.subplot(1,2,1)
    pyplot.title('Original image')
    pyplot.axis('off')
    pyplot.imshow(oryg)
    pyplot.subplot(1,2,2)
    pyplot.xlabel('Pixels value')
    pyplot.ylabel('Number of pixels')
    pyplot.title('Histogram of original image')
    for i,color in enumerate(color):
        hist = cv2.calcHist([oryg],[i],None,[256],[0,256])
        pyplot.plot(hist,color = color)
        pyplot.xlim([0,256])
    pyplot.show()

def edges(image):
    oryg = readImage(image)
    edges = cv2.Canny(oryg,100,200)
    title1 = 'Original image'
    title2 = 'Canny edge detecting image output'
    cmap = 'viridis'
    ploting2(oryg,title1,edges,title2,cmap)

def scanCodes(image):
    oryg = readImage(image)
    image = oryg.copy()
    decodedObjects = pyzbar.decode(oryg)
    if decodedObjects != '':
        for obj in decodedObjects:
            print('Type of code : ', obj.type)
            print('Data on code: ', obj.data,'\n')
    else:
      print('Unable to read code. Please try another image.')
    for decodedObject in decodedObjects:
        points = decodedObject.polygon
        if len(points) > 4 :
            hull = cv2.convexHull(np.array([point for point in points]))#, dtype=np.float32))
            hull = list(map(tuple, np.squeeze(hull)))
        else :
            hull = points;

        n = len(hull)

        for j in range(0,n):
            cv2.line(image, hull[j], hull[ (j+1) % n], (255,0,0), 3)
    title1 = 'Original image'
    title2 = 'Image with surrounded barcode'
    cmap = 'viridis'
    ploting2(oryg,title1,image,title2,cmap)

def make_file(select_video,frame_rate):
    path = 'digitization_1/files/{}.mp4'.format(select_video)
    cap = cv2.VideoCapture(path)
    size = int(cap.get(3)),int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'VP08')
    out = cv2.VideoWriter('frame_rates.webm', fourcc, frame_rate, size)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            out.write(frame)
        else:
            break
    files.download('frame_rates.webm')

##### Ćwiczenie z HDR

def show_oryg(img_1, img_2, img_3):

    pyplot.figure(figsize=[12.0, 8.0])
    pyplot.subplot(1,3,1)
    pyplot.imshow(img_1)
    pyplot.axis('off')
    pyplot.subplot(1,3,2)
    pyplot.imshow(img_2)
    pyplot.axis('off')
    pyplot.subplot(1,3,3)
    pyplot.imshow(img_3)
    pyplot.axis('off')
    pyplot.suptitle('Original taken images', x=0.5, y=0.65, fontsize=21)

def plotingHDR(hdr_out, hdr_type):
    pyplot.figure(figsize=[7.5, 5.0])
    pyplot.title('Image created by {} HDR type.'.format(hdr_type.upper()))
    pyplot.imshow(hdr_out)
    pyplot.axis('off')
    pyplot.show()

def loadFilesHDR(image):
    img_fn = ['digitization_1/files/{}{}.png'.format(image,str(num)) for num in range(1,4)]
    img_list = [cv2.imread(fn) for fn in img_fn]
    img_list_ok = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in img_list]
    return img_list_ok;

def hdr(image, hdr_type):
    img_list_ok = loadFilesHDR(image)
    img_1, img_2, img_3 = img_list_ok
    show_oryg(img_1, img_2, img_3)
    exposure_times = np.array([50.0, 11.0, 3.0],dtype=np.float32) #, dtype=np.float32
    tonemap1 = cv2.createTonemap(gamma=2.5)

    if hdr_type == 'robertson':
        merge_robertson = cv2.createMergeRobertson()
        hdr_robertson = merge_robertson.process(img_list_ok, times=exposure_times.copy())
        res_robertson = tonemap1.process(hdr_robertson.copy())
        hdr_out = np.clip(res_robertson*255, 0, 255).astype('uint8')


    elif hdr_type == 'mertens':
        merge_mertens = cv2.createMergeMertens()
        res_mertens = merge_mertens.process(img_list_ok)
        hdr_out = np.clip(res_mertens*255, 0, 255).astype('uint8')

    elif hdr_type == 'debevec':
        time = exposure_times
        merge_debevec = cv2.createMergeDebevec()
        hdr_debevec = merge_debevec.process(img_list_ok, times=time.copy())
        res_debevec = tonemap1.process(hdr_debevec.copy())
        hdr_out = np.clip(res_debevec*255, 0, 255).astype('uint8')

    else:
        print('Błąd funkcji "hdr".')

    plotingHDR(hdr_out, hdr_type)

## Experimenting with frame rates
Please try to change the default frame rate.<br>

<font color=red><b>Mention that the dropdown menu will be visible only after the appropriate code cell is run</b></font>

From dropdown menu choose one of example videos and set number of frames per second on slide bar. A video file will be downloaded automatically. Please open it and observe changes. Each change will generate another file.<br><br>

In [ ]:
#@title Exercise { display-mode: "form" }
interact(make_file, select_video = [('Falling fruit','lemon'),('View on street','cariage'),('Reading people','glasses')], frame_rate = IntSlider(min=5, max=150, step=5, continuous_update=False));


## Experiment With Color Spaces

Please try to change the default colour space for loaded image. Please observe the relation between the image colours and colour space.<br>

At first click inside code-block and click Run button in navigate panel.

<font color=red><b>What do you mean by "advantage red colour"?</b></font><br>
<font color=blue><b>Chciałem zaznaczyć, że na tym zdjęciu przeważa kolor czerwony (w kanale RED jest najwięcej pikseli o wysokich wartościach)</b></font>

In [ ]:
#@title Exercise 1 - run cell and experiment with prepared images { display-mode: "form" }
interact(cs, image=[('Advantage red color', 'digitization_1/files/red.jpg'),('Advantage green color', 'digitization_1/files/green.jpg'),('Advantage blue color', 'digitization_1/files/blue.jpg'), ('Random shapes', 'digitization_1/files/rgb.jpg')], set_channel=[('RGB color space','RGB'),('R','R'),('G','G'),('B','B'),('YCbCr color space','YCbCr'),('Y','Y'),('Cb','Cb'),('Cr','Cr'),('YUV color space','YUV'),('Y','Y_'),('U','U'),('V','V'),('HSV','HSV'),('H','H'),('S','S'),('V','V_')]);

In [ ]:
#@title Exercise 2 - run cell and paste http address of your image { display-mode: "form" }
interact(cs, image='https://thumbs.dreamstime.com/b/multi-colours-strips-texture-design-165353569.jpg', set_channel=[('RGB color space','RGB'),('R','R'),('G','G'),('B','B'),('YCbCr color space','YCbCr'),('Y','Y'),('Cb','Cb'),('Cr','Cr'),('YUV color space','YUV'),('Y','Y_'),('U','U'),('V','V'),('HSV','HSV'),('H','H'),('S','S'),('V','V_')]);

## Experimenting With Picture Histogram
This demo uses the 2-D Histogram in Image Processing to calculate the histograms of R, G, and B values.

Click inside code-block and click Run button in navigate panel.

In [ ]:
#@title Exercise 1 - experiment with pre-loaded images { display-mode: "form" }
interact(histogram, image=[('Advantage red color', 'digitization_1/files/red.jpg'),('Advantage green color', 'digitization_1/files/green.jpg'),('Advantage blue color', 'digitization_1/files/blue.jpg'),('Overexposure image','digitization_1/files/overexp.jpg'),('Underexposure image','digitization_1/files/underexp.jpg')]);

In [ ]:
#@title Exercise 2 - run cell and paste http address of your image { display-mode: "form" }
interact(histogram, image='https://thumbs.dreamstime.com/b/multi-colours-strips-texture-design-165353585.jpg');

## Experimenting with Edge Detection
The Canny method is applied to find the edges of objects in the input images

Click inside code-block and click Run button in navigate panel.

In [ ]:
#@title Exercise 1 - run cell and experiment with pre-loaded images { display-mode: "form" }
interact(edges, image=[('Random shapes', 'digitization_1/files/shapes.jpg'),('Advantage red color', 'digitization_1/files/red.jpg'),('Advantage green color', 'digitization_1/files/green.jpg'),('Advantage blue color', 'digitization_1/files/blue.jpg')]);

In [ ]:
#@title Exercise 2 - run cell and paste http address of your image { display-mode: "form" }
interact(edges, image='https://thumbs.dreamstime.com/b/multi-colours-strips-texture-design-165353532.jpg');

## Experimenting with Barcode Recognition

In [ ]:
#@title Exercise 1 - Experiment with pre-loaded images { display-mode: "form" }
interact(scanCodes, image=[('QR 1','digitization_1/files/qrcode1.jpg'),('QR 2','digitization_1/files/qrcode2.jpg'),('QR 3','digitization_1/files/qrcode3.jpg'),('BAR 1','digitization_1/files/barcode1.jpg'),('BAR 2','digitization_1/files/barcode2.jpg'),('BAR 3','digitization_1/files/barcode3.jpg')]);

In [ ]:
#@title Exercise 2 - run cell and paste http address of your image { display-mode: "form" }
interact(scanCodes, image='https://barcodesgonewild.com/wp-content/uploads/2014/05/Axe-barcode-1.jpeg');

# Digitization - Advanced Topics in Digital Photography
*Michał Grega, Mikołaj Leszczuk, Jakub Nawała, Grzegorz Pasterczyk*

## Purpose 
Purpose of this laboratory is to present the RAW processing workflow for digital images and the HDR technique.

## Prerequisites
* Basics of digital photography
* Basics of image formats

## RAW Processing
*All photographs are © Michał Grega unless stated otherwise.*

## What is RAW and why to use it?
RAW is a file format used for storing the information on the image taken by the digital camera. It is not an image format. It contains raw (unprocessed) data stored by the physical sensor (radiometric data). Apart from that, a RAW file may contain additional metadata on:
* Make and model of the camera,
* Physical properties of the sensor,
* Exposure and camera settings,
* Lens settings,
* A highly compressed .jpg thumbnail of the image.

Please note, that the camera sensor is most commonly not a pixel matrix (see Fig. 1). It is a CMOS or a CCD (charge-coupled device) sensor covered by a filter (see Fig. 2). Underneath this filter, there is an array of photosensitive subpixels, which do not have to be even of rectangular shape. Therefore in order to convert the radiometric data to an image detailed information on the sensor geometry must be available for the software algorithm.

<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1ptl8YS1g5EqjQ7_Y0oK3WL1naqTKkOEf" width="500">

Fig. 1. Sensor layouts (Wikipedia)

<img src="https://drive.google.com/uc?export=view&id=1O_Y89x3GTTPQ9J8Qqte8In9-DQ4cC43y" width="500">

Fig. 2. Bayer colour filter (do you see anything unexpected?) (Wikipedia)</center></div></body></html>

## How it differs from .jpg or .tiff?
A RAW image captured by a camera is an uncompressed and unprocessed raw measurement of light. It is commonly referred to as a digital negative, as it serves a similar purpose as a traditional film negative. A .jpg or .tiff image produced by the camera is that digital negative processed (developed) on the fly by the camera built-in software. The software most typically conducts a set of automated operations:
1. develop the raw image (knowing the physical properties of the sensor),
* enhance the resulting image (by applying contrast and colour correction and sharpening),
* apply additional correction algorithms (e.g. red-eye reduction),
* compress the image to the desired format (lossy .jpg or lossless .tiff).

## What are the benefits of RAW shooting?
The most profound and important benefit is that a photographer retains full control of the creation, correction and compression processes. All the adjustments can be made by hand and tuned in order to achieve the desired effect.<br> 
Moreover, RAW files offer much better input for post-processing, as the state-of-the-art sensors store (digitize) the data at 14 bits per colour per pixel. It means that a raw image can hold 214 shades per colour, meaning 242 total colours. A .jpg file typically saves 8 bits per colour, meaning 28 shades per colour resulting in 224 colours. In short, the RAW format offers better colour fidelity **(18 orders of magnitude greater than .jpg)**, much higher dynamic range (High Dynamic Range imaging will be explained further on) and more data for further corrections.<br>
What are the drawbacks of RAW shooting?
* The visual quality of an unenhanced RAW file is not satisfying as no corrections are applied,
* RAW images are of large size,
* RAW images require tedious manual development and correction. 

## RAW processing workflow
`
Disclaimer:
Photography is an art and thus slips away from scientific definition. Moreover, it is controversial how much post-processing (a.k.a) “photoshopping” is allowed to a professional. Photographic agencies and photographic competitions have strict rules that define what is allowed and what is not.
`
The RAW processing workflow consists of several steps - all described below. Each photographer usually creates his/her own workflow by adding or removing some of the steps. It is important to sustain the order of the steps, as there is a logic behind them (e.g. sharpening has to be done prior to development).
1. **Cropping and straightening** – a selection of a composition of the image
<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=18wRyA5FDFnTovss-rfxsa_0ae3M0YICC" width="500"/>
<p>Before and after straightening </p></center></div></body></html>

2. **Exposure correction** – done in order to correct for over- or underexposed images. Due to the physical characteristics of the sensor a rule of the thumb is that it is better to shoot under-. rather than, overexposed images as it is easier to compensate for underexposure. The most useful tool is the luminosity histogram. A well-exposed photo covers the whole dynamic range and fills the whole histogram. An under- or overexposed photo shows clipping in (respectively) low or high values.
<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1mtBmPzGXFJ2bZQIxpNljrwYL15BOAePe" width="500"/>
<p>A histogram of an underexposed image </p>
<img src="https://drive.google.com/uc?export=view&id=189mjDgxRqx2B96T7HwAYOishQ5rZAwme" width="500"/>
<p>A histogram of an overexposed image</p>
<p>(gray area - total luminosity; red, green and blue curves - luminosity for each RGB channel)</p>
<img src="https://drive.google.com/uc?export=view&id=1i8B-2KZ4hwshZPAJXDEuDU5zA0FMCX-Z" width="500"/>
<img src="https://drive.google.com/uc?export=view&id=1ZqMOJaDyymCdckE8OsBKXco0pK-uf6Aj" width="500"/>
<p>Before and after exposure correction. Notice the histogram.</p>
</center></div></body></html>

More advanced software allows for a software-based increase in dynamic range (i.e. the increase in an end-to-end distance between extreme pixel values). Software algorithm detects the under- or overexposed parts of the image and enhances them instead of modifying the whole image.
<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1dChZqeL3rcYU_KuTXov6sJqQ6_z-esvv" width="600"/>
<img src="https://drive.google.com/uc?export=view&id=1EDdxReNJgOr6fWlnY2PlMAwDCpiooSxB" width="600"/>
<img src="https://drive.google.com/uc?export=view&id=1ngDySajgzyMw0a7jdLrSidXeevu-Vky2" width="600"/>
<p> Exposure correction using overall exposure and software HDR. Notice the contrast between foreground and background.</p>
</center></div></body></html>

3. **Contrast correction** – increases the contrast in an image. Images captured in the RAW format appear to be flat and not vibrant. That is due to the lack of contrast correction. Contrast is the difference between the brightest and darkest pixel in the image. While it is easy to define, there are many algorithms that aim at improving contrast by maintaining the general luminosity and colours of the image. 
<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1P2nc32N7ok7vxQhGKGhJ976SJsJj8dat" width="600"/>
<img src="https://drive.google.com/uc?export=view&id=1GWIqYDuiVI9sE_nPkkCig6qnuSTGX-zX" width="600"/>
<p> Contrast correction </p>
</center></div></body></html>

4. **Colour correction** – shooting an image in given conditions may cause the colours to be distorted. Especially the type of light (sunlight vs artificial) makes the colours unnatural. For example, shooting in artificial incandescent light causes images to be unnaturally warm (due to the high amount of infrared radiation). On the other hand, shooting in full sunlight on high altitudes causes photographs to be unnaturally cool (because of the high amount of UV radiation). It can be compensated for using white balance compensation.

<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1VdqrzX6ojjP5w0xrDm3l109Is9bv9ASu" width="600"/>
<img src="https://drive.google.com/uc?export=view&id=1lVOL2GqRPdqUK3pBkoilNPkXJ69KKSCs" width="600"/>
<p> White balance compensation – notice the clipping on a histogram in the red channel. </p>
</center></div></body></html>

5. **Sharpening and detail** – allows to sharpen the image and remove unnecessary artefacts. Among those are spots caused by dirt on the sensor (or lenses) and noise generated by the sensor itself.

<html><body><div><center>
<img src="https://drive.google.com/uc?export=view&id=1xKAT3EMW0kE0TfePj4p6Ty_g7izh-kx6" width="600"/>
<p> Sharpening and noise reduction </p>
</center></div></body></html>

6. **Development** – allows converting the image to the target format and colour space.


## RAW processing exercise
1. You can use any RAW processing software you wish. Note, however, that the paid software usually offers a more intuitive interface and more advanced algorithms. If you own a DSLR (Digital Single-Lens Reflex) camera you probably got a copy of the manufacturer’s software. Other (costly) solutions are Adobe Photoshop with Lightroom or Capture One (for the use in the laboratory you have to download a version from https://www.phaseone.com/en/Download.aspx).<br>You can also use (free) http://rawtherapee.com/.<br>Examples shown in the previous section were prepared using Capture One Pro, which offers a free 30-day trial.
But also you can see exercise below to see how RAW processing works on 3 examples.
2. Download example RAW files (see the “RAW Examples” folder accompanying this instruction).
3. Develop these RAW files into *.jpg images for web publishing trying to achieve the best visual result. Correct the composition, exposure and colours of the image. Apply sharpening and the correct developmental recipe. Observe what happens when you use high values of the corrections for sharpness, software HDR, exposition. There is a saying for beginners “Set up your sliders in a position that makes your photograph look good and then reduce all by half”.

## HDR Imaging
As you might have noticed, one of the most challenging scenes is those with high contrast between shadowy and bright regions. Each optical device, including the human eye, has a dynamic range. A dynamic range is a difference measured in EV units between the darkest and brightest part of the image that shows detail. Increase of one EV unit represents a situation where the amount of light is doubled. A human eye and a modern DSLR camera sensor have a dynamic range of approx. 14 EV (called “stops”). It means that we can double the amount of light 14 times between the brightest and darkest part of the image and still see detail.<br>We can control which part of the scene is covered by our EV range by adjusting the shutter speed, aperture or ISO value of the sensor. We, however, cannot increase this range.

## What is HDR?
HDR, High Dynamic Range, is a photographing technique, in which a set of images is made with different camera setups. Each photograph covers a limited dynamic range, but a combination of the photographs covers a higher dynamic range resulting in an HDR photograph.<br>Of course, the display or printout also has a limited dynamic range, thus a mapping from the wider to the narrower dynamic range has to be done. This is referred to as “tone mapping”.<br>For an exemplary usage of the HDR technique, please take a look at the example below.

<center><img src='https://drive.google.com/uc?export=view&id=1a6b01LPYeYMQnNGcYSUY62Zhp69RV0QI' width="600"/>
<p>HDR input images. Note, that it was impossible to get details both on the bright (sky) and dark (shadows) areas in any single photo.</p><br><br>
<img src='https://drive.google.com/uc?export=view&id=1Q3sgo3W0GjGu4i9_4os3qFj5-we1HKw4' width="600"/>
<p>HDR result. Notice, that it looks unnatural, as it shows a higher dynamic range than a human eye is able to process.</p></center>

## High Dynamic Range (HDR) exercise
Here you can see how HDR works with examples. In exercise used 3 different HDR methods:
* Debevec
* Robertson
* Martens
Each of them you can check with 3 images taken with different exposure time. 

Here you can see oryginal taken pictures with average time of exposure. Below them, choose from drop-down menu picture you would like to see after HDR conversion.
<table style="width:80%">
  <tr>
    <th style="text-align:center">Yosemite</th>
    <th style="text-align:center">Garden</th>
    <th style="text-align:center">Mount</th>
  </tr>
  <tr>
    <td><img src="https://drive.google.com/uc?export=view&id=1NothOGCG0HYhn6iyAo2w2bbPnO63zvma"/></td>
    <td><img src="https://drive.google.com/uc?export=view&id=15SvrRxCbOHYHeLQ8t-b0jw6RK2IRQVSX"/></td>
    <td><img src="https://drive.google.com/uc?export=view&id=1ITqflKJ3T0I2BPcRQAIo-H-d_M46cjKh"/></td></td>
  </tr>
</table>


In [ ]:
#@title Exercise  - run cell and experiment with pre-loaded images { display-mode: "form" }
interact(hdr, image=['yosemite','garden','mount'], hdr_type=['robertson','mertens','debevec']);